In [1]:
import cv2
import os
import numpy as np

In [2]:
def save_video_frames(video_path, output_folder, frames_to_get):
    cap = cv2.VideoCapture(video_path)
    for i in frames_to_get:
        ret, frame = cap.read()
        if not ret:
            break
        frame_path = os.path.join(output_folder, f"frame_{i:04d}.png")
        cv2.imwrite(frame_path, frame)
    cap.release()

In [5]:
video_path = 'eye.mp4'
output_folder = 'video_frames'
save_video_frames(video_path, output_folder, [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200])

In [6]:
def merge_images_to_video(image_folder, output_video_path, frame_rate=30):
    image_files = [f for f in os.listdir(image_folder) if f.endswith('.png')]
    image_files.sort()
    first_image_path = os.path.join(image_folder, image_files[0])
    img = cv2.imread(first_image_path)
    height, width, layers = img.shape
    video_writer = cv2.VideoWriter(output_video_path, cv2.VideoWriter_fourcc(*'mp4v'), frame_rate, (width, height))
    for image_file in image_files:
        image_path = os.path.join(image_folder, image_file)
        img = cv2.imread(image_path)
        video_writer.write(img)
    video_writer.release()

In [7]:
image_folder = 'video_frames'
output_video_path = 'output_video.mp4'
merge_images_to_video(image_folder, output_video_path, frame_rate=30)

In [11]:
def capture_frames_from_webcam(output_folder):
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        return
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        cv2.imshow('Webcam Feed', frame)
        image_path = os.path.join(output_folder, f"frame_{len(os.listdir(output_folder)):04d}.png")
        cv2.imwrite(image_path, frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

In [12]:
output_folder = 'captured_frames'
capture_frames_from_webcam(output_folder)

In [11]:
def chroma_keying(video_path_subject, video_path_background, output_path):
    cap_subject = cv2.VideoCapture(video_path_subject)
    cap_background = cv2.VideoCapture(video_path_background)
    width = int(cap_subject.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap_subject.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap_subject.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    while True:
        ret_subject, frame_subject = cap_subject.read()
        ret_background, frame_background = cap_background.read()
        if not ret_subject or not ret_background:
            break
        hsv_frame = cv2.cvtColor(frame_subject, cv2.COLOR_BGR2HSV)
        mask = cv2.inRange(hsv_frame, np.array([40, 40, 40]), np.array([80, 255, 255]))
        inverted_mask = cv2.bitwise_not(mask)
        subject_part = cv2.bitwise_and(frame_subject, frame_subject, mask=inverted_mask)
        background_part = cv2.bitwise_and(frame_background, frame_background, mask=mask)
        composite_frame = cv2.add(subject_part, background_part)
        out.write(composite_frame)
    cap_subject.release()
    cap_background.release()
    out.release()

In [12]:
subject_video_path = 'subject_video.mp4'
background_video_path = 'background_video.mp4'
output_video_path = 'composite_video.mp4'
chroma_keying(subject_video_path, background_video_path, output_video_path)